In [3]:
# Import necessary libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import flatten
import numpy as np
import random
import json
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf


nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

class Training:
    def __init__(self):
        # read and load the intent file
        data_file = open('intents1.json').read()
        self.intents = json.loads(data_file)['intents']
        self.ignore_words = list("!@#$%^&*?")
        self.process_data()

    def process_data(self):
        # fetch patterns and tokenize them into words
        self.pattern = list(map(lambda x: x["patterns"], self.intents))
        self.words = list(map(word_tokenize, flatten(self.pattern)))
        # fetch classes i.e. tags and store in documents along with tokenized patterns
        self.classes = flatten([[x["tag"]] * len(y) for x, y in zip(self.intents, self.pattern)])
        self.documents = list(map(lambda x, y: (x, y), self.words, self.classes))
        # lower case and filter all the symbols from words
        self.words = list(map(str.lower, flatten(self.words)))
        self.words = list(filter(lambda x: x not in self.ignore_words, self.words))

        # lemmatize the words and sort the class and word lists
        self.words = list(map(lemmatizer.lemmatize, self.words))
        self.words = sorted(list(set(self.words)))
        self.classes = sorted(list(set(self.classes)))

    def train_data(self):
        # initialize and set analyzer=word as we want to vectorize words not characters
        cv = CountVectorizer(tokenizer=lambda txt: txt.split(), analyzer="word", stop_words=None)
        # create the training sets for the model
        training = []
        for doc in self.documents:
            # lower case and lemmatize the pattern words
            pattern_words = list(map(str.lower, doc[0]))
            pattern_words = ' '.join(list(map(lemmatizer.lemmatize, pattern_words)))

            # train or fit the vectorizer with all words
            # and transform into one-hot encoded vector
            vectorize = cv.fit([' '.join(self.words)])
            word_vector = vectorize.transform([pattern_words]).toarray().tolist()[0]

            # create output for the respective input
            # output size will be equal to the total number of classes
            output_row = [0] * len(self.classes)
            output_row[self.classes.index(doc[1])] = 1

            cvop = cv.fit([' '.join(self.classes)])
            out_p = cvop.transform([doc[1]]).toarray().tolist()[0]

            # store vectorized word list along with its class
            training.append([word_vector, output_row])

        # shuffle training sets to avoid the model training on the same data again
        random.shuffle(training)
        training = np.array(training, dtype=object)
        train_x = list(training[:, 0])  # patterns
        train_y = list(training[:, 1])  # classes
        return train_x, train_y

    def build(self):
        # load the data from train_data function
        train_x, train_y = self.train_data()

        # Create a Sequential model with 3 layers.
        model = Sequential()
        # input layer with a latent dimension of 128 neurons and ReLU activation function
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))  # Dropout to avoid overfitting
        # second layer with the latent dimension of 64 neurons
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        # fully connected output layer with softmax activation function
        model.add(Dense(len(train_y[0]), activation='softmax'))
       # Compile model with legacy Stochastic Gradient Descent
        sgd = tf.keras.optimizers.legacy.SGD(learning_rate=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



        # fit the model with training input and output sets
        hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=10, verbose=1)
        # save the model and words, classes which can be used for prediction.
        model.save('NewModel.h5')
        pickle.dump({'words': self.words, 'classes': self.classes, 'train_x': train_x, 'train_y': train_y},
                    open("training_data", "wb"))

# train the model
Training().build()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Developer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Developer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/200
10/10 [==============================] - 1s 5ms/step - loss: 4.0231 - accuracy: 0.0100
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 3.9763 - accuracy: 0.0200
Epoch 3/200
10/10 [==============================] - 0s 5ms/step - loss: 3.9775 - accuracy: 0.0600
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 3.9067 - accuracy: 0.1200
Epoch 5/200
10/10 [==============================] - 0s 5ms/step - loss: 3.8803 - accuracy: 0.1600
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 3.7837 - accuracy: 0.1500
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 3.7809 - accuracy: 0.1600
Epoch 8/200
10/10 [==============================] - 0s 4ms/step - loss: 3.6847 - accuracy: 0.1800
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 3.7162 - accuracy: 0.1400
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 3.6031 - accuracy: 0.2100
Epoch 11/